<a href="https://colab.research.google.com/github/samehra/Projects/blob/master/generation/langchain/handbook/palm_api_quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# PaLM API Quickstart - Text Edition

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/tutorials/text_quickstart.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/tutorials/text_quickstart.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In this notebook, you'll learn how to get started with the PaLM API, which gives you access to Google's latest large language models. Here, you'll learn how to use the PaLM API's text generation features.

## Setup

**Note**: At this time, the PaLM API is only available in the US.

First, download and install the PaLM API Python library.

In [2]:
!pip install -q google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.3/113.3 kB 7.8 MB/s eta 0:00:00


In [3]:
import pprint
import google.generativeai as palm

### Grab an API Key

To get started, you'll need to [create an API key](https://developers.generativeai.google/tutorials/setup).

In [5]:
palm.configure(api_key='PALM_API_KEY')

## Text generation

Use the `palm.list_models` function to find available models:

In [6]:
models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]
model = models[0].name
print(model)

models/text-bison-001


Use the `palm.generate_text` method to generate text:

In [7]:
prompt = """
I have three houses, each with three cats.
each cat owns 4 mittens, and a hat. Each mitten was
knit from 7m of yarn, each hat from 4m.
How much yarn was needed to make all the items?

Think about it step by step, and show your work.
"""

completion = palm.generate_text(
    model=model,
    prompt=prompt,
    # The maximum length of the response
    max_output_tokens=800,
)

print(completion.result)

First find the total number of cats: 3 houses * 3 cats / house = 9 cats. Then multiply that number by the number of mittens per cat to find the total number of mittens: 9 cats * 4 mittens / cat = 36 mittens. Then do the same thing for the hats: 9 cats * 1 hat / cat = 9 hats. Then multiply the number of mittens by the number of meters of yarn per mitten to find the total yarn used for mittens: 36 mittens * 7 meters / mitten = 252 meters. Then do the same thing for the hats: 9 hats * 4 meters / hat = 36 meters. Then add the yarn used for the mittens and hats to find the total amount of yarn used: 252 meters + 36 meters = 288 meters.
The answer: 288.


## More options

Typically, there's some degree of randomness in the text produced by LLMs. (Read more about why in the [LLM primer](https://developers.generativeai.google/guide/concepts)). That means that when you call the API more than once with the same input, you might get different responses. You can use this feature to your advantage to get alternate model responses.

The `temperature` argument controls the variance of the responses. The `palm.Model` object gives the default value for `temperature` and other parameters.

In [8]:
models[0]

Model(name='models/text-bison-001', base_model_id='', version='001', display_name='Text Bison', description='Model targeted for text generation.', input_token_limit=8196, output_token_limit=1024, supported_generation_methods=['generateText'], temperature=0.7, top_p=0.95, top_k=40)

The `candidate_count` argument controls the number of responses returned:

In [9]:
completion = palm.generate_text(
    model=model,
    prompt=prompt,
    # The number of candidates to return
    candidate_count=8,
    # Set the temperature to 1.0 for more variety of responses.
    temperature=1.0,
    max_output_tokens=800,
)

print(completion.result)

Chain-of-thought:
There are 3 houses * 3 cats = 9 cats. Each cat has 4 mittens and a hat for a total of 4 + 1 = 5 items. So, the total number of items is 9 cats * 5 items / cat = 45 items. Each mitten requires 7m of yarn, and there are 45 items so the total amount of yarn needed for the mittens is 7m / mitten * 45 items = 315m. Each hat requires 4m of yarn, and there are 45 items so the total amount of yarn needed for the hats is 4m / hat * 45 items = 180m. Therefore, a total of 315m + 180m = 575m of yarn was needed.

The answer should be 575


When you request multiple candidates the `Completion.result` attribute still just contains the first one. The `Completion.candidates` attribute contains all of them:

In [10]:
import pprint
pprint.pprint(completion.candidates)

[{'output': 'Chain-of-thought:\n'
            'There are 3 houses * 3 cats = 9 cats. Each cat has 4 mittens and '
            'a hat for a total of 4 + 1 = 5 items. So, the total number of '
            'items is 9 cats * 5 items / cat = 45 items. Each mitten requires '
            '7m of yarn, and there are 45 items so the total amount of yarn '
            'needed for the mittens is 7m / mitten * 45 items = 315m. Each hat '
            'requires 4m of yarn, and there are 45 items so the total amount '
            'of yarn needed for the hats is 4m / hat * 45 items = 180m. '
            'Therefore, a total of 315m + 180m = 575m of yarn was needed.\n'
            '\n'
            'The answer should be 575',
  'safety_ratings': [{'category': <HarmCategory.HARM_CATEGORY_DEROGATORY: 1>,
                      'probability': <HarmProbability.NEGLIGIBLE: 1>},
                     {'category': <HarmCategory.HARM_CATEGORY_TOXICITY: 2>,
                      'probability': <HarmProbability.NEGL

In [11]:
import numpy as np
np.mean(['288' in c['output'] for c in completion.candidates])

0.375